In [5]:
# import dependencies 
from config import secrets 
import requests
import pandas as pd
import pprint 

api_key = secrets.congress_api_key
req_format = '&format=json'

### Request Congressional Sessions

In [9]:
congress_req = f'https://api.congress.gov/v3/congress?api_key={api_key}{req_format}'
cognress_json = requests.get(congress_req)
# cognress_json.json()['congresses']

[{'endYear': '2024',
  'name': '118th Congress',
  'sessions': [{'chamber': 'House of Representatives',
    'endDate': '2024-01-03',
    'number': 1,
    'startDate': '2023-01-03',
    'type': 'R'},
   {'chamber': 'Senate',
    'endDate': '2024-01-03',
    'number': 1,
    'startDate': '2023-01-03',
    'type': 'R'},
   {'chamber': 'Senate', 'number': 2, 'startDate': '2024-01-03', 'type': 'R'},
   {'chamber': 'House of Representatives',
    'number': 2,
    'startDate': '2024-01-03',
    'type': 'R'}],
  'startYear': '2023',
  'url': 'https://api.congress.gov/v3/congress/118?format=json'},
 {'endYear': '2022',
  'name': '117th Congress',
  'sessions': [{'chamber': 'House of Representatives',
    'endDate': '2022-01-03',
    'number': 1,
    'startDate': '2021-01-03',
    'type': 'R'},
   {'chamber': 'Senate',
    'endDate': '2022-01-03',
    'number': 1,
    'startDate': '2021-01-03',
    'type': 'R'},
   {'chamber': 'House of Representatives',
    'endDate': '2023-01-03',
    'number'

In [26]:
chambers = pd.json_normalize(cognress_json.json()['congresses'], 
                             record_path = 'sessions',
                             meta = ['name', 'startYear', 'endYear'])
chambers['congress_num'] = chambers.name.str.extract(r'([0-9]{2,3})')
chambers.sample(5)

,chamber,endDate,number,startDate,type,name,startYear,endYear,congress_num
77,Senate,1985-12-20,1,1985-01-03,R,99th Congress,1985,1986,99
69,Senate,1989-11-22,1,1989-01-03,R,101st Congress,1989,1990,101
74,House of Representatives,1988-10-22,2,1988-01-25,R,100th Congress,1987,1988,100
33,Senate,2007-12-19,1,2007-01-04,R,110th Congress,2007,2008,110
14,House of Representatives,2019-01-03,2,2018-01-03,R,115th Congress,2017,2018,115


In [35]:
chambers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   chamber       80 non-null     object
 1   endDate       78 non-null     object
 2   number        80 non-null     int64 
 3   startDate     80 non-null     object
 4   type          80 non-null     object
 5   name          80 non-null     object
 6   startYear     80 non-null     object
 7   endYear       80 non-null     object
 8   congress_num  80 non-null     object
dtypes: int64(1), object(8)
memory usage: 5.8+ KB


### Request Members of each Session

In [56]:
chamber_nums = chambers[chambers.startYear >= '2000'].congress_num.unique()

reps = pd.DataFrame()

for c in chamber_nums[::-1]:
  member_req = f'https://api.congress.gov/v3/member/congress/{c}?api_key={api_key}{req_format}'
  member_json = requests.get(member_req)
  temp_df = pd.json_normalize(member_json.json()['members'], 
                              record_path = ['terms', 'item'],
                              meta = ['bioguideId', 'partyName', 'state', 'updateDate'])
  temp_df['chamber_num'] = c
  reps = pd.concat([reps, temp_df], ignore_index = True)

In [66]:
member_json.json()['members']

[{'bioguideId': 'G000579',
  'depiction': {'attribution': 'Congressional Pictorial Directory',
   'imageUrl': 'https://www.congress.gov/img/member/115_rp_wi_8_gallagher_mike_200.jpg'},
  'district': 8,
  'name': 'Gallagher, Mike',
  'partyName': 'Republican',
  'state': 'Wisconsin',
  'terms': {'item': [{'chamber': 'House of Representatives',
     'endYear': 2024,
     'startYear': 2017}]},
  'updateDate': '2024-07-06T18:36:15Z',
  'url': 'https://api.congress.gov/v3/member/G000579?format=json'},
 {'bioguideId': 'B001297',
  'depiction': {'attribution': 'Congressional Pictorial Directory',
   'imageUrl': 'https://www.congress.gov/img/member/115_rp_co_4_buck_ken_200.jpg'},
  'district': 4,
  'name': 'Buck, Ken',
  'partyName': 'Republican',
  'state': 'Colorado',
  'terms': {'item': [{'chamber': 'House of Representatives',
     'endYear': 2024,
     'startYear': 2015}]},
  'updateDate': '2024-07-06T18:36:15Z',
  'url': 'https://api.congress.gov/v3/member/B001297?format=json'},
 {'biogui

In [57]:
reps.sample(20)

,chamber,endYear,startYear,bioguideId,partyName,state,updateDate,chamber_num
166,House of Representatives,NaN,2013,K000381,Democratic,Washington,2024-06-08T18:40:22Z,114
248,House of Representatives,2023.0,2011,C001084,Democratic,Rhode Island,2024-06-10T13:28:06Z,118
26,House of Representatives,2019.0,1997,S000250,Republican,Texas,2024-06-08T18:40:21Z,108
0,House of Representatives,1997.0,1991,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,107
186,House of Representatives,NaN,2015,P000610,Democratic,Virgin Islands,2024-06-08T18:40:22Z,115
252,House of Representatives,NaN,2023,B001318,Democratic,Vermont,2024-06-08T18:40:22Z,118
106,House of Representatives,NaN,2001,W000795,Republican,South Carolina,2024-06-08T18:40:21Z,111
142,House of Representatives,NaN,2009,C001078,Democratic,Virginia,2024-06-08T18:40:22Z,113
10,House of Representatives,NaN,1995,D000399,Democratic,Texas,2024-06-08T18:40:21Z,107
91,House of Representatives,NaN,1997,S000510,Democratic,Washington,2024-06-08T18:40:22Z,111


In [59]:
reps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   chamber      258 non-null    object 
 1   endYear      48 non-null     float64
 2   startYear    258 non-null    int64  
 3   bioguideId   258 non-null    object 
 4   partyName    258 non-null    object 
 5   state        258 non-null    object 
 6   updateDate   258 non-null    object 
 7   chamber_num  258 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 16.3+ KB


In [70]:
reps[(reps.chamber == 'Senate')]

,chamber,endYear,startYear,bioguideId,partyName,state,updateDate,chamber_num
1,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,107
23,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,108
45,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,109
67,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,110
89,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,111
113,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,112
135,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,113
157,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,114
179,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,115
200,Senate,NaN,1997,R000122,Democratic,Rhode Island,2024-06-19T12:42:22Z,116
